In [1]:
import os
import tensorflow as tf
from tensorflow import keras

In [2]:
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11490434/11490434 [==============================] - 1s 0us/step


In [3]:
#definicja modelu
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Trening modelu z zapisem punktów kontrolnych (checkpoints)

In [4]:
checkpoints_path = "training_1/cp.ckpt"
checkpoints_dir = os.path.dirname(checkpoints_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoints_path,
                                                 save_weights_only = True,
                                                 verbose = 1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])

Epoch 1/10
24/32 [=====================>........] - ETA: 0s - loss: 1.3089 - sparse_categorical_accuracy: 0.6302
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 2s 21ms/step - loss: 1.1474 - sparse_categorical_accuracy: 0.6740 - val_loss: 0.7077 - val_sparse_categorical_accuracy: 0.7960
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.4337 - sparse_categorical_accuracy: 0.8940
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - loss: 0.4315 - sparse_categorical_accuracy: 0.8930 - val_loss: 0.5111 - val_sparse_categorical_accuracy: 0.8460
Epoch 3/10
26/32 [=======================>......] - ETA: 0s - loss: 0.2898 - sparse_categorical_accuracy: 0.9315
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.2836 - sparse_categorical_accuracy: 0.9350 - val_loss: 0.4915 - val_sparse_categorical_accuracy: 0.8400
Epoch 4/10
25/32 [=======

In [5]:
os.listdir(checkpoints_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [6]:
model = create_model()
loss, acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"niewytrenowany model -> dopasowanie: {100*acc:.2f}%")

32/32 - 0s - loss: 2.3199 - sparse_categorical_accuracy: 0.1070 - 178ms/epoch - 6ms/step
niewytrenowany model -> dopasowanie: 10.70%


In [7]:
model.load_weights(checkpoints_path)
loss, acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"odtworzony model -> dopasowanie: {100*acc:.2f}%")

32/32 - 0s - loss: 0.4122 - sparse_categorical_accuracy: 0.8690 - 80ms/epoch - 3ms/step
odtworzony model -> dopasowanie: 86.90%


In [8]:
checkpoints_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoints_dir = os.path.dirname(checkpoints_path)

batch_size = 32
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoints_path,
                                                 save_weights_only = True,
                                                 save_freq = 5*batch_size,
                                                 verbose = 1)

model = create_model()
model.save_weights(checkpoints_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size = batch_size,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback],
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [9]:
os.listdir(checkpoints_dir)

['checkpoint',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0040.ckpt.index']

In [10]:
latest = tf.train.latest_checkpoint(checkpoints_dir)
latest

'training_2/cp-0050.ckpt'

In [11]:
model = create_model()
model.load_weights(latest)
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"odtworzony model --> dopasowanie: {100*acc:.2f}%")

32/32 - 0s - loss: 0.5085 - sparse_categorical_accuracy: 0.8710 - 251ms/epoch - 8ms/step
odtworzony model --> dopasowanie: 87.10%


In [12]:
model.save_weights('./checkpoints/my_checkopint')
model = create_model()
model.load_weights('./checkpoints/my_checkopint')
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"odtworzony model --> dopasowanie: {100*acc:.2f}%")

32/32 - 0s - loss: 0.5085 - sparse_categorical_accuracy: 0.8710 - 173ms/epoch - 5ms/step
odtworzony model --> dopasowanie: 87.10%


In [13]:
#zapis modelu
model = create_model()
model.fit(train_images,train_labels,epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 4ms/step - loss: 1.1322 - sparse_categorical_accuracy: 0.6820
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4094 - sparse_categorical_accuracy: 0.8810
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2752 - sparse_categorical_accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2058 - sparse_categorical_accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1419 - sparse_categorical_accuracy: 0.9690
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [14]:
new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [15]:
loss,acc = new_model.evaluate(test_images,test_labels,verbose=2)
print(f"odtworzony model --> dopasowanie: {100*acc:.2f}%")
print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4209 - sparse_categorical_accuracy: 0.8630 - 190ms/epoch - 6ms/step
odtworzony model --> dopasowanie: 86.30%
32/32 [==============================] - 0s 2ms/step
(1000, 10)
